## Generating an OTU table and imposing QC filters, based on QIIME2 "Atacama soil microbiome" tutorial and the Divisive Amplicon Denoising Algorithm (DADA2)
https://docs.qiime2.org/2017.2/tutorials/atacama-soils/

http://benjjneb.github.io/dada2/

Last class, we demultiplexed our reads (grouped them by sample) and looked at the quality scores for the forward and reverse reads of our sequences. You should be able to pull the quality score graphs up here:

In [ ]:
!qiime tools view demux-qualities.qzv

(Remember that you can stop the notebook running the view command by pressing the square stop button at the top of the page. This has the same effect as entering CTRL+C in the terminal.)

The region that we chose to sequence is ~bp long, and so has lots of overlap between the forward and reverse reads. The first thing you might have noticed is that the reverse read tends to be of lower quality. That is an artifact of the Illumina sequencing platform. The other thing you'll notice is that the quality tends to tail off towards the end. That's also standard. Finally, you might see that the beginning of the reads looks a little iffy too. Fortunately, by trimming the reads and combining the forward and reverse reads, we can create single, full-length sequences that are high-quality.

The next command packages a lot of steps into one, so it's important to know what it's doing.

In [1]:
!qiime dada2 --help

Usage: qiime dada2 [OPTIONS] COMMAND [ARGS]...

  Plugin website: http://benjjneb.github.io/dada2/

  Getting user support: To get help with DADA2, post to the DADA2 issue
  tracker: https://github.com/benjjneb/dada2/issues

  Citing this plugin: DADA2: High-resolution sample inference from Illumina
  amplicon data. Benjamin J Callahan, Paul J McMurdie, Michael J Rosen,
  Andrew W Han, Amy Jo A Johnson, Susan P Holmes. Nature Methods 13, 581–583
  (2016) doi:10.1038/nmeth.3869.

Options:
  --help  Show this message and exit.

Commands:
  denoise-paired  Denoise and dereplicate paired-end sequences
  denoise-single  Denoise and dereplicate single-end sequences
  plot-qualities  Plot positional qualitites


This tells us the various commands that DADA2 can run, and where to find out more. In our case, we have paired-end sequences, so we want to know what that command does:

In [3]:
!qiime dada2 denoise-paired --help

Usage: qiime dada2 denoise-paired [OPTIONS]

  This method denoises paired-end sequences, dereplicates them, and filters
  chimeras.

Options:
  --i-demultiplexed-seqs PATH     Artifact:
                                  SampleData[PairedEndSequencesWithQuality]
                                  [required]
                                  The paired-end demultiplexed
                                  sequences to be denoised.
  --p-trunc-len-f INTEGER         [required]
                                  Position at which forward read
                                  sequences should be truncated due to
                                  decrease in quality. This truncates the 3'
                                  end of the of the input sequences, which
                                  will be the bases that were sequenced in the
                                  last cycles. Reads that are shorter than
                                  this value will be discarded.
  --p-trunc-len-r 

Now you can see that this command does a lot at once. It requires our demultiplexed input file that we created last tutorial "--i-demultiplexed-seqs demux.qza", and it also requires a length at which we will truncate our sequences. Because this data was generated using 150bp paired ends sequencing, anything longer than 150 is suspect. So, we'll use 150 as the cutoff for our forward and reverse reads  "--p-trunc-len-f 150 --p-trunc-len-r 150".

From looking at the quality scores earlier, we saw that the sequence quality dips off at the end. Fortunately matching up the forward and reverse reads will help take care of that, to some extent. However, to deal with the lower sequence quality at the very beginning of the reads, we might just proactively cut out the first few base pairs of those sequences "--p-trim-left-f 10  --p-trim-left-r 10". Of course, you could choose any number here.

We can also include a parameter to set the maximum expected errors <http://www.drive5.com/usearch/manual/exp_errs.html> that we accept in a given read. We will use 1.0 here - the default is 2.0, so if you don't specify a value, it will use 2.0.

dada2 also counts the number of each type of read, which requires that it compare all reads to each other. This is different from previous "OTU-clustering" algorithims, which we will discuss in class. Briefly, it considers the possible errors in each sequence, and compares all sequences to each other, which allows it to predict the likelihood that a given sequence is "real" or not. This means that it removes all "singletons" - sequences that are only present once across all samples, which are very hard to distingish from sequencing errors. It also removed "chimeras" - erroneous combinations of two different true sequences.

After performing most of the previous QC functions, it starts with all the sequences in one group, and compares all the other sequences to the most abundant sequence in the whole dataset. If any sequences fall below a certain threshold of similarity to that central sequence, then a second sequence is chosen as a second centre, and all sequences are compared to these two centres. This is repeated iteratively, adding more and more centres, until no sequences fall outside of the threshold. In the current version of QIIME2, this is set at 1e-40.

Finally, we want to tell QIIME where to put and what to call the output. It will give us an OTU table and the sequences associated with each OTU "--o-table table --o-representative-sequences rep-seqs".

Let's give it a spin! This will also take a few minutes.

In [14]:
!qiime dada2 denoise-paired --i-demultiplexed-seqs demux.qza --p-trim-left-f 10 --p-trim-left-r 10 --p-trunc-len-f 150 --p-trunc-len-r 150 --p-max-ee 2 --o-table table --o-representative-sequences rep-seqs 

Saved FeatureTable[Frequency] to: table.qza
Saved FeatureData[Sequence] to: rep-seqs.qza


And for a quick summary of the results of this function, we can use the following commands:

In [15]:
!qiime feature-table summarize --i-table table.qza --o-visualization table.qzv --m-sample-metadata-file sample-metadata.tsv
!qiime feature-table tabulate-seqs --i-data rep-seqs.qza --o-visualization rep-seqs.qzv

Saved Visualization to: table.qzv
Saved Visualization to: rep-seqs.qzv


Let's take a look at the table:

In [16]:
!qiime tools view table.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

And we can visualize the sequences too:

In [18]:
!qiime tools view rep-seqs.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.